In [7]:
import pandas as pd
import codecs
import csv
import json
import numpy as np
import re
import matplotlib
import string

import nltk
from langdetect import detect

csv_filename = 'C:/Users/Robin/Dropbox/Travaux_&_Rapports_Stages/UbicomLab-GATech/MeToo/code/Data_tweets/tweetsbalancetonporc.csv'
json_filename='C:/Users/Robin/Dropbox/Travaux_&_Rapports_Stages/UbicomLab-GATech/MeToo/code/Data_tweets/tweetsbalancetonporc-13_10-11-8.json'
json_filename2='C:/Users/Robin/Dropbox/Travaux_&_Rapports_Stages/UbicomLab-GATech/MeToo/code/Data_tweets/tweetsbalancetonporc-13_10-11-8 - Copie.json' 
json_metoo='C:/Users/Robin/Dropbox/Travaux_&_Rapports_Stages/UbicomLab-GATech/MeToo/code/data-metoo/metoo.json'
df = pd.read_json(json_filename, orient="columns")
dfc=pd.read_json(json_filename2, orient="columns")
df_metoo=pd.read_json(json_metoo,orient='colums')
df_metoo_c=pd.read_json(json_metoo,orient='colums')

In [ ]:
df.head()

# Cleaning data

In [10]:
step1(df) #extract link, pictures and create a special column
step2(df) # extract the # and the @ and creta a special column
#step3(df_metoo) # delete punctiation, and figures.
sup_lang(df,dfc,"fr")# delete tweet which are not in the language define
step5(df) # delete the column languange and put everything in lowercase
step6(df,'btp_clean_tweet_with_punctiation.json') #save the data under another json file

il y a  fullname     5478
id           5478
likes        5478
replies      5478
retweets     5478
text         5478
timestamp    5478
user         5478
url          5478
url_image    5478
hashtag      5478
mention      5478
langue       5478
dtype: int64 qui vont être supprimés
76010 76010
76010 70532


### 1) Extract url link from tweet and create a special column

In [9]:

######################################################################################### STEP 1
#Function that enables to delete link in a text
remove_url=lambda x: re.sub(find_url, '', x)
remove_img=lambda x: re.sub(find_img,' ',x)
find_url = re.compile(r'(?:http|ftp|https)://(?:[\w_-]+(?:(?:\.[\w_-]+)+))(?:[\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?')
find_img=re.compile(r'(pic.twitter.com/[^\s]+)')
def step1(df):
    #create a colum for url link
    l=int(len(df['id']))

    url_tab=[]
    tweet_url=[]
    img_tab=[]
    for text in df['text']:
        url_tab.append(find_url.findall(text))
        img_tab.append(find_img.findall(text))
    #print(url_tab)
    tweet_url=[]*len(url_tab)
    tweet_img=[]*len(img_tab)       
    for i in range(len(url_tab)):
        if len(url_tab[i])==0 or url_tab[i][0]==0:
            url_tab[i]=[""]
        tweet_url.append(url_tab[i][0])
        if len(img_tab[i])==0 or img_tab[i][0]==0:
            img_tab[i]=[""]
        tweet_img.append(img_tab[i][0])
     
    df['url']=tweet_url#add a column
    df.loc[:,['url']] = df['url'].values

    df['url_image']=tweet_img#add a column
    df.loc[:,['url_image']] = df['url_image'].values

    df['text']=df['text'].map(remove_url) #We apply this function to the serie 'text'
    df['text']=df['text'].map(remove_img)
#find image link
#exemple: tweet numeros: 9516
#find_img=re.compile(r'(pic).(twitter).(com)/(([^\s]+))')
#print(find_img.findall(dfc['text'][9516]),dfc['text'][9516])
######################################################################################### STEP 2

#create a column for #

find_hastag = re.compile(r'\B(\#[a-zA-Z]+\b)(?!;)')
revome_hastag=lambda x: re.sub(find_hastag, '', x)
find_mention = re.compile(r'\B(\@[a-zA-Z]+\b)(?!;)')
revome_mention=lambda x: re.sub(find_mention, '', x)

def step2(df):
    has_tab=[]
    men_tab=[]
    for text in df['text']:
        h=find_hastag.findall(text)
        k=find_mention.findall(text)
        has_tab.append([h])
        men_tab.append([k])
    
    #print(url_tab)
    hashtag_tab=[]*len(has_tab)
    mention_tab= []*len(men_tab)
    for i in range(len(has_tab)):
        if len(has_tab[i])==0 or has_tab[i][0]==0:
            has_tab[i]=[""]
        hashtag_tab.append(str(has_tab[i][0]))

    for i in range(len(men_tab)):
        if len(men_tab[i])==0 or men_tab[i][0]==0:
            men_tab[i]=[""]
        mention_tab.append(str(men_tab[i][0]))
    df['hashtag']=hashtag_tab#add a column
    df.loc[:,['hashtag']] = df['hashtag'].values
    
    df['mention']=mention_tab#add a column
    df.loc[:,['mention']] = df['mention'].values
    #supression des # et des @
    df['text']=df['text'].map(revome_hastag)
    df['text']=df['text'].map(revome_mention)
    
    #Convertir les mention et les hashtag en string
    #for i in range(len(df['hashtag'])):
     #   df['hashtag'][i].to_string
    df['hashtag']=df['hashtag'].str.replace('[', '')
    df['hashtag']=df['hashtag'].str.replace(']','')
    
    df['mention']=df['mention'].str.replace('[', '')
    df['mention']=df['mention'].str.replace(']','')
    
    
    #print(df['hashtag'][9982],df['text'][9982])
    #print(p2.findall(df['text'][1]))
    #txt=df['text'][5]
    #print(f(txt),txt)

######################################################################################### STEP 3

#We replace !,.?»« by spaces
def step3(df):
    g=df['text']#creating a copy for the test
    
    df['text'] = df['text'].str.replace(',', ' ')
    df['text'] = df['text'].str.replace('!', ' ')
    df['text'] = df['text'].str.replace('.', ' ')
    df['text'] = df['text'].str.replace('?', ' ')
    df['text'] = df['text'].str.replace('»', '')
    df['text'] = df['text'].str.replace('«', '')
    
    #We remove all special caracteres : !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
    remove_ponctuation = lambda x: ''.join(ch for ch in x if ch not in set(string.punctuation))
    
    df['text']=df['text'].map(remove_ponctuation)
    
    #print(k[5],df['text'][5])
    
    #remove RT
    df['text'] = df['text'].str.replace('RT', ' ')
    #remove number
    find_numbers = re.compile(r'\d+')
    revome_numbers=lambda x: re.sub(find_numbers, '', x)
    df['text']=df['text'].map(revome_numbers)
    
######################################################################################### STEP SUP_LANG
def sup_lang(df,dfc,lg):
    #print(nltk.word_tokenize(df['text'][33399]))
    detect_lg=lambda x: 1 if detect(x)==lg else 0
    nb_mot=lambda x:len(nltk.word_tokenize(x))
    l=len(df['text'])
    #print(len(df))
    langue=[]
    k=0
    for i in range(l):
        
        if nb_mot(df['text'].iloc[i])<=2: # If there is less than two word, we consider that the method is ineficient so, we keep it
            langue.append('1')
            k+=1
        else:
            try:
                langue.append(detect_lg(df['text'].iloc[i]))
            except:
                langue.append('1')

    df['langue']=langue
    #print(langue[:100])
    print ("il y a ",df[df['langue']==0].agg('count'),"qui vont être supprimés")
    print(len(dfc),len(df))
    #df=df.drop(df.index[df.text.map(detect_fr)==0]) 
    #langue=lambda x:len(nltk.word_tokenize(x))
    df=df.drop(df.index[df.langue==0])
    print(len(dfc),len(df))
#df_metoo[df_metoo['langue']==0].agg('count')
#df[df['langue']==0].agg('count')
#sup_lang(df,dfc,'fr')
#sup_lang(df_metoo,df_metoo_c,'en')



def step5(df):
    #import nltk
    #print(nltk.word_tokenize(df['text'][33399]))
    #nb_mot=lambda x:len(nltk.word_tokenize(x))
    # df['text'] = df['text'].str.replace(',', ' ')
    
    #Everithing lower cse
    df['hashtag'] = df['hashtag'].map(lambda x: x.lower())
    df['mention'] = df['mention'].map(lambda x: x.lower())
    del df['langue']

######################################################################################### STEP 6
def step6(df,filename):
    df.to_json(filename)
    #df2=pd.read_json(filename, orient="columns")

# Discovering Pandas

In [ ]:
#Have a look on the first 5 lines
print(df.head())
#Provide basics stats about this column
df.likes.describe()
#Select all the Tweets with more than 500 retweets
df[df['retweets']>500]
#count the number of tweet with this attributes
(df[df['retweets']>500]).agg('count')
print(df['text'][0])
df[df['retweets']>500]['retweets']
(df[df['retweets']>500]).agg('count')